In [1]:
# ! pip install ucimlrepo

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

# Implementation 

In [ ]:
from initialization_builder import InitializationBuilder
from activation_builder import ActivationBuilder


class Layer:
    __slots__ = [
        "nodes_in",
        "nodes_out",
        "weights",
        "weights_gradient",
        "biases",
        "biases_gradient",
        "activation",
        "a",
        "z",
    ]

    def __init__(
        self,
        nodes_in,
        nodes_out,
        activation="sigmoid",
        weight_initialization="he",
        bias_initialization="zero",
    ):

        self.nodes_in = nodes_in
        self.nodes_out = nodes_out

        self.weights = InitializationBuilder.get_initialization(
            weight_initialization, nodes_out, nodes_in
        )

        self.weights_gradient = InitializationBuilder.get_initialization(
            "zero", nodes_out, nodes_in
        )

        self.biases = InitializationBuilder.get_initialization(
            bias_initialization, nodes_out, 1
        )
        self.biases_gradient = InitializationBuilder.get_initialization(
            "zero", nodes_out, 1
        )

        self.activation = ActivationBuilder.get_activation(activation)

    def forward(self, a):
        self.z = np.dot(self.weights, a) + self.biases
        self.a = self.activation.activation(self.z)
        return self.a

    def report_layer(self, layer_num):
        return (
            f"Layer number {layer_num}\nWeights\n{self.weights}\nbiases\n{self.biases}"
        )

In [ ]:
from cost_function_builder import CostFunctionBuilder


class NeuralNetwork:
    __slots__ = [
        "layers",
        "optimizer",
        "cost_function",
        "layer_sizes",
    ]

    def __init__(
        self,
        cost_function="mse",
    ):
        self.layers = []
        self.layer_sizes = []
        self.cost_function = CostFunctionBuilder().build_cost_function(cost_function)

    def add_layer(self, layer: Layer):
        if not self.layer_sizes:
            self.layer_sizes = [layer.nodes_in]
        else:
            assert (
                layer.nodes_in == self.layer_sizes[-1]
            ), f"Output in previous layer doesn't match input in this layer"
        self.layer_sizes.append(layer.nodes_out)
        self.layers.append(layer)

    def _forward(self, x: np.ndarray):
        if isinstance(x, pd.DataFrame) or isinstance(x, pd.Series):
            x = x.to_numpy()
            if len(x.shape) == 1:
                x = x.reshape(-1, 1)
        a = x.T
        for layer in self.layers:
            a = layer.forward(a)
        return a

    def predict(self, x: np.ndarray):
        return self._forward(x).T

    def predict_class(self, x: np.ndarray):
        return np.argmax(self.predict(x), axis=1, keepdims=True)

    def flatten_weights_and_biases(self):
        weights_and_biases = []
        for layer in self.layers:
            weights_and_biases.append(layer.weights.flatten())
            weights_and_biases.append(layer.biases.flatten())
        return np.concatenate(weights_and_biases)

    def deflatten_weights_and_biases(self, solution):
        for layer in self.layers:
            layer.weights = solution[: layer.weights.size].reshape(layer.weights.shape)
            solution = solution[layer.weights.size :]
            layer.biases = solution[: layer.biases.size].reshape(layer.biases.shape)
            solution = solution[layer.biases.size :]

    def calculate_cost(self, x: np.ndarray, y: np.ndarray):
        base_cost = self.cost_function.cost(self.predict(x), y)
        return base_cost

In [ ]:
class Evolutionary_Algorithm:
    __slots__ = [
        "mutation_rate",
        "crossover_rate",
        "number_of_generations",
        "population_size",
        "function_to_optimize",
        "population",
        "problem_dim",
        "hall_of_fame",
    ]

    def __init__(
        self,
        mutation_rate=0.7,
        crossover_rate=0.7,
        number_of_generations=50,
        population_size=100,
    ):
        self.population_size = population_size
        self.number_of_generations = number_of_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.function_to_optimize = None
        self.population = None
        self.problem_dim = None
        self.hall_of_fame = []

    def _generate_population(self, n_genes):
        self.population = np.random.uniform(-10, 10, (self.population_size, n_genes))

    def _mutation(self, individual):
        for i in range(len(individual)):
            if np.random.rand() < self.mutation_rate:
                individual += np.random.normal(0, 1, self.problem_dim)
        return individual

    def _crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.problem_dim - 1)
            child1 = np.concatenate(
                (parent1[:crossover_point], parent2[crossover_point:])
            )
            child2 = np.concatenate(
                (parent2[:crossover_point], parent1[crossover_point:])
            )
            return child1, child2
        return parent1, parent2

    def _evaluate_population(self):
        fitness = np.zeros(len(self.population))
        for i, individual in enumerate(self.population):
            fitness[i] = self.function_to_optimize(*individual)
        self.hall_of_fame.append(np.argsort(fitness)[: int(self.population_size * 0.1)])
        return fitness

    def _tournament_selection(self, fitness):
        fitness = 1 / fitness
        probabilities = fitness / np.sum(fitness)
        return self.population[
            np.random.choice(
                range(len(self.population)),
                p=probabilities,
                size=self.population_size - len(self.hall_of_fame[-1]),
                replace=True,
            )
        ]

    def _select_individual(self, n=1):
        return self.population[
            np.random.choice(range(self.population_size), size=n, replace=False)
        ]

    def _visualize_individual(self, individual):
        print(individual)

    def optimize(self, function_to_optimize, silent=False):
        # initialize the population
        self.function_to_optimize = function_to_optimize
        self.problem_dim = function_to_optimize.__code__.co_argcount
        self._generate_population(self.problem_dim)
        best_evaluation_in_iteration = []
        best_solution_in_iteration = []

        # main algorithm loop
        for generation in range(self.number_of_generations):

            # crossover
            children = np.zeros(
                (2 * len(range(0, self.population_size, 2)), self.problem_dim)
            )
            for i in range(0, self.population_size, 2):
                parent1, parent2 = self._select_individual(n=2)
                children[i], children[i + 1] = self._crossover(parent1, parent2)
            self.population = np.vstack([self.population, children])

            # mutation
            # we mutate every individual in the population not random one
            mutated = np.zeros((self.population_size, self.problem_dim))
            for i in range(self.population_size):
                individual = self._select_individual()
                mutated[i] = self._mutation(individual)
            self.population = np.vstack([self.population, mutated])

            # evaluate the population and log the best solution
            fitness = self._evaluate_population()
            if not silent:
                print(
                    f"iter: {generation}, best: {np.min(fitness)} for {['%.3f' % n for n in self.population[self.hall_of_fame[-1][0]]]}"
                )

            best_evaluation_in_iteration.append(np.min(fitness))
            best_solution_in_iteration.append(self.population[self.hall_of_fame[-1][0]])

            # create new population
            self.population = np.vstack(
                [
                    self.population[self.hall_of_fame[-1]],
                    self._tournament_selection(fitness),
                ]
            )

        return best_evaluation_in_iteration, best_solution_in_iteration

# Testing methods 

### MPG

In [5]:
# fetch dataset
auto_mpg = fetch_ucirepo(id=9)

# data (as pandas dataframes)
X = auto_mpg.data.features
y = auto_mpg.data.targets

### Iris

In [ ]:
# fetch dataset
iris = fetch_ucirepo(id=53)

# data (as pandas dataframes)
X = iris.data.features
y = iris.data.targets

### Multimodal large

In [ ]:
multimodal_large_train = pd.read_csv(
    "https://raw.githubusercontent.com/nizwant/miowid/main/data/regression/multimodal-large-training.csv"
)
multimodal_large_test = pd.read_csv(
    "https://raw.githubusercontent.com/nizwant/miowid/main/data/regression/multimodal-large-test.csv",
)

In [ ]:
mean = multimodal_large_train.mean()
std = multimodal_large_train.std()
multimodal_large_train = (multimodal_large_train - mean) / std
multimodal_large_test = (multimodal_large_test - mean) / std